# Duvidas

1. Qual é o threshold da distribuiçao para podermos considerar um valor como "long tail" e podermos agrupar com um valor de "other"? 
    - no caso de geo_cities testou-se com percentil 60 e passámos a ter 100 e poucos valores distintos. 
    - ja na page_path_level_3 ao percentil 75 ainda sao perto de 200 valores

# File and libraries

In [49]:
import pandas as pd
import numpy as np


filepath=r'data/google_merch_store_raw_merge_america.csv'



file_tag = "ga4_merch_store"

# DSLabs functions

In [50]:
%run "scripts/dslabs_functions.py"


# data functions

In [51]:
%run "scripts/data_functions.py"


data_functions lodaded


# north america filtering df creation

- use if you only have full df version and not northern america filtered file

In [52]:
# data=pd.read_csv(r'google_merch_store_raw_merge.csv')

# data=data[data['geo_sub_continent']=='Northern America']

# data.to_csv('google_merch_store_raw_merge_america.csv',index=False)

# data.to_csv('google_merch_store_raw_merge_america_compressed.csv',index=False, compression='gzip')



# Load and sampling

In [53]:
# test_data=True
test_data=False


# Define a function to sample 10% from each group
def sample_per_day(group, fraction=0.1):
    return group.sample(frac=fraction)


if test_data==True:

    data=pd.read_csv(filepath)
    

    data['event_timestamp'] = pd.to_datetime(data['event_timestamp'], unit='us', utc=True)
    data['event_date'] = pd.to_datetime(data['event_date'], infer_datetime_format=True)
 

    # Apply the sampling to each group (grouped by event_date) 1%
    data = data.groupby('event_date').apply(lambda x: sample_per_day(x, 0.01)).reset_index(drop=True)

   

else:
    data=pd.read_csv(filepath)
    

    data['event_timestamp'] = pd.to_datetime(data['event_timestamp'], unit='us', utc=True)
    data['event_date'] = pd.to_datetime(data['event_date'], infer_datetime_format=True)

    # 10% sample
    data = data.groupby('event_date').apply(lambda x: sample_per_day(x, 0.1)).reset_index(drop=True)
    

    

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405866 entries, 0 to 405865
Data columns (total 48 columns):
 #   Column                           Non-Null Count   Dtype              
---  ------                           --------------   -----              
 0   event_date                       405866 non-null  datetime64[ns]     
 1   session_id                       405866 non-null  int64              
 2   user_pseudo_id                   405866 non-null  float64            
 3   event_name                       405866 non-null  object             
 4   event_timestamp                  405866 non-null  datetime64[ns, UTC]
 5   page_location                    405866 non-null  object             
 6   page_title                       404763 non-null  object             
 7   device_category                  405866 non-null  object             
 8   device_mobile_brand_name         405866 non-null  object             
 9   device_mobile_model_name         405866 non-null  object   

In [54]:
data.shape

(405866, 48)

# class target column creation


- we want to classify if that hit is from a returning or new user.
- Due to web analytics tracking particularities like cookie acceptance we prefer to consider returning users as users that are on their 3rd or larger session number
- In this case, new user (ga_session_number <=2) will be 0 and returning user will be more than 2 (ga_session_number > 2)
- session number column shall be removed afterwards as well

In [55]:

data['returning_user'] = data['ga_session_number'].apply(lambda x: 0 if x <= 2 else 1)


data=data.drop(['ga_session_number'],axis=1) # now we do not need it anymore. remove it



In [56]:
target = "returning_user"

values = data[target].value_counts(normalize=True) 
print(values)

returning_user
0    0.754266
1    0.245734
Name: proportion, dtype: float64


# unbalance dataset

let us umbalance to have 10% as returning users
UPDATE: NOT NEEDED

In [57]:
# # Separate the majority (0) and minority (1) classes
# df_majority = data[data['returning_user'] == 0]
# df_minority = data[data['returning_user'] == 1]

# # Calculate the number of minority rows needed to make a 90/10 split
# # Let total_rows be the total number of rows after undersampling
# total_rows = len(df_majority) / 0.9  # 90% majority, 10% minority
# desired_minority_count = int(total_rows * 0.1)  # 10% of the total should be minority

# # Downsample the minority class to the desired number of rows
# df_minority_downsampled = df_minority.sample(n=desired_minority_count, random_state=42)

# # Combine the majority class with the downsampled minority class
# df_imbalanced = pd.concat([df_majority, df_minority_downsampled])

# # Shuffle the combined dataset
# data = df_imbalanced.sample(frac=1, random_state=42).reset_index(drop=True)

# # Check the new class distribution to verify the 90/10 split
# print(data['returning_user'].value_counts(normalize=True) * 100)

returning_user
0    90.000147
1     9.999853
Name: proportion, dtype: float64


# column drop

### low value or high null count columns

In [58]:
data=data.drop(['debug_mode','device_is_limited_ad_tracking','device_mobile_marketing_name','geo_metro','traffic_source_name','page_referrer','entrances'],axis=1)

### ecommerce specific columns

In [59]:
data=data.drop([
 'ecommerce_total_item_quantity', 
 'ecommerce_purchase_revenue',            
 'ecommerce_shipping_value',              
 'ecommerce_tax_value',                   
 'ecommerce_unique_items',               
 'ecommerce_transaction_id',              
 'item_id',                               
 'item_name',                             
 'item_brand',                            
 'item_variant',                          
 'item_category',                         
 'price',                                 
 'quantity',                              
 'item_revenue',                          
 'item_list_index',                       
 'promotion_name',
 'geo_continent',
 'geo_sub_continent'],axis=1)

In [60]:
summary5 = data.describe(include="all")

summary5

,event_date,session_id,user_pseudo_id,event_name,event_timestamp,page_location,page_title,device_category,device_mobile_brand_name,device_mobile_model_name,...,device_web_info_browser,device_web_info_browser_version,geo_country,geo_region,geo_city,traffic_source_medium,traffic_source_source,session_engaged,engagement_time_msec,returning_user
count,340145,3.401450e+05,3.401450e+05,340145,340145,340145,339139,340145,340145,340145,...,340145,340145,340145,340145,340145,340145,340145,310361.000000,2.534880e+05,340145.000000
unique,NaN,NaN,NaN,17,NaN,1032,478,3,8,10,...,6,13,2,62,325,6,5,NaN,NaN,NaN
top,NaN,NaN,NaN,view_item,NaN,https://shop.googlemerchandisestore.com/,Home,desktop,Apple,Chrome,...,Chrome,87.0,United States,California,(not set),organic,google,NaN,NaN,NaN
freq,NaN,NaN,NaN,123691,NaN,31278,55271,198100,144389,94121,...,231935,125505,291314,61399,146712,118910,124003,NaN,NaN,NaN
mean,2020-12-14 05:31:30.707786240,5.003594e+09,2.673831e+08,NaN,2020-12-14 17:31:52.658177280+00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.907276,1.208460e+04,0.099999
min,2020-11-01 00:00:00,5.455500e+04,1.001303e+06,NaN,2020-11-01 00:00:04.579566+00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000e+00,0.000000
25%,2020-11-26 00:00:00,2.539015e+09,5.815923e+06,NaN,2020-11-26 05:10:53.949998080+00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.690000e+03,0.000000
50%,2020-12-11 00:00:00,5.004306e+09,2.233245e+07,NaN,2020-12-11 14:39:56.469096960+00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,5.546000e+03,0.000000
75%,2021-01-04 00:00:00,7.514050e+09,6.131530e+07,NaN,2021-01-04 19:04:46.331791872+00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.272500e+04,0.000000
max,2021-01-31 00:00:00,9.999568e+09,9.985207e+09,NaN,2021-01-31 23:56:53.265848+00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,3.659287e+06,1.000000


# drop specific event_names

We want to classify a user by its interactions with the website so we want to exclude some actions that may also be biased by incorrect ga4 tracking namely:
- session_start
- first_visit
- click (low event count)
- view_item_list (may not be triggered by user interaction)


In [61]:
if test_data==True:
    data['event_name'].value_counts()

In [62]:
# List of values to drop
events_to_drop = ['session_start', 'first_visit','click','view_item_list']

# drop events from list
data = data[~data['event_name'].isin(events_to_drop)]

if test_data==True:
    data['event_name'].value_counts()

# replace (not set) with null

we will handle these later but these are actually null values

In [63]:
data.replace('(not set)', np.nan, inplace=True)

# Engagement time msec
- https://support.google.com/analytics/answer/11109416?hl=en
- is it null?

In [64]:
data['engagement_time_msec'] = data['engagement_time_msec'].fillna(0)

# geo columns

## geo_region and geo_city

In [65]:
null_region_cities_df=data.groupby(['geo_region']).agg(
    null_count=('geo_city', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()

# will retrieve the most populated cities of these regions to use as fill method
region_cities_with_nulls = null_region_cities_df[null_region_cities_df['null_count'] > 0]['geo_region'].tolist()


region_cities_with_nulls

['California',
 'Florida',
 'Texas',
 'New Jersey',
 'New York',
 'Pennsylvania',
 'Michigan',
 'Ohio',
 'Georgia',
 'Illinois',
 'Ontario',
 'Massachusetts',
 'North Carolina',
 'Virginia',
 'Maryland',
 'Quebec',
 'Connecticut',
 'Missouri',
 'Wisconsin',
 'Washington',
 'Indiana',
 'Colorado',
 'Minnesota',
 'South Carolina',
 'British Columbia',
 'Utah',
 'Tennessee',
 'Alberta',
 'Alabama',
 'Louisiana',
 'Oregon',
 'Kentucky',
 'Mississippi',
 'Iowa',
 'Arizona',
 'Arkansas',
 'Oklahoma',
 'Kansas',
 'New Hampshire',
 'Delaware',
 'West Virginia',
 'Idaho',
 'Manitoba',
 'Rhode Island',
 'Montana',
 'Nova Scotia',
 'Maine',
 'Saskatchewan',
 'New Mexico',
 'North Dakota',
 'Alaska',
 'New Brunswick',
 'Vermont',
 'Nebraska',
 'Nevada',
 'Hawaii',
 'Prince Edward Island',
 'Newfoundland and Labrador',
 'Wyoming',
 'South Dakota']

## geo mapping

In [66]:
most_populated_cities = {
    'California': 'Los Angeles',
    'Florida': 'Jacksonville',
    'Texas': 'Houston',
    'New Jersey': 'Newark',
    'New York': 'New York City',
    'Pennsylvania': 'Philadelphia',
    'Michigan': 'Detroit',
    'Ohio': 'Columbus',
    'Illinois': 'Chicago',
    'Georgia': 'Atlanta',
    'Massachusetts': 'Boston',
    'North Carolina': 'Charlotte',
    'Virginia': 'Virginia Beach',
    'Maryland': 'Baltimore',
    'Connecticut': 'Bridgeport',
    'Indiana': 'Indianapolis',
    'Wisconsin': 'Milwaukee',
    'Missouri': 'Kansas City',
    'Washington': 'Seattle',
    'Colorado': 'Denver',
    'Minnesota': 'Minneapolis',
    'South Carolina': 'Charleston',
    'Utah': 'Salt Lake City',
    'Tennessee': 'Nashville',
    'Alabama': 'Birmingham',
    'Louisiana': 'New Orleans',
    'Oregon': 'Portland',
    'Kentucky': 'Louisville',
    'Mississippi': 'Jackson',
    'Iowa': 'Des Moines',
    'Arkansas': 'Little Rock',
    'Arizona': 'Phoenix',
    'Oklahoma': 'Oklahoma City',
    'Kansas': 'Wichita',
    'Delaware': 'Wilmington',
    'West Virginia': 'Charleston',
    'New Hampshire': 'Manchester',
    'Idaho': 'Boise',
    'Montana': 'Billings',
    'Maine': 'Portland',
    'New Mexico': 'Albuquerque',
    'North Dakota': 'Fargo',
    'Rhode Island': 'Providence',
    'Nebraska': 'Omaha',
    'Alaska': 'Anchorage',
    'Vermont': 'Burlington',
    'Hawaii': 'Honolulu',
    'Nevada': 'Las Vegas',
    'Wyoming': 'Cheyenne',
    'South Dakota': 'Sioux Falls'
}

data['geo_city'] = data['geo_city'].fillna(data['geo_region'].map(most_populated_cities))


## geo city long tail conversion

In [67]:
# Step 1: Aggregate events by 'page_location' and 'page_path_level_3'
df_cities_agg = data.groupby(['geo_city']).agg(
    total_events=('event_timestamp', 'count'),
).sort_values('total_events', ascending=False).reset_index()

# Step 2: Calculate the desired percentile of total events to consider long tail
event_count_percentile = df_cities_agg['total_events'].quantile(0.65)

# Step 3: Create a mask for long tail pages
long_tail_mask = df_cities_agg['total_events'] < event_count_percentile

# Step 4: Create a dictionary to map original page_path_level_3 to "other" (long tail)
long_tail_mapping = dict(zip(df_cities_agg['geo_city'], 
                             df_cities_agg['geo_city'].where(~long_tail_mask, "Other")))


# Step 5: Replace the values directly in the original DataFrame
data['geo_city'] = data.apply(
    lambda row: long_tail_mapping.get(row['geo_city'], row['geo_city']),
    axis=1
)


# event timestamp treatment

- need to convert to local time from different us timezones

## timezone from regions and create local_event_timestamp

In [68]:
geo_region_list = data['geo_region'].tolist()
distinct_geo_regions = list(set(geo_region_list))

print(distinct_geo_regions)

['Tennessee', 'New Jersey', 'New York', 'Michigan', 'South Carolina', 'Rhode Island', 'Vermont', 'California', 'Quebec', 'Nova Scotia', 'Massachusetts', 'Idaho', 'Connecticut', 'Maine', 'Wyoming', 'Oklahoma', 'Illinois', 'Mississippi', 'Utah', 'Colorado', 'West Virginia', 'Arizona', 'Alaska', 'Montana', 'District of Columbia', 'British Columbia', 'Texas', 'Saskatchewan', 'Alabama', 'Maryland', 'Iowa', 'Nevada', 'Wisconsin', 'New Mexico', nan, 'Missouri', 'Alberta', 'South Dakota', 'Kentucky', 'Arkansas', 'Manitoba', 'Pennsylvania', 'Hawaii', 'Kansas', 'North Dakota', 'Ohio', 'Washington', 'Prince Edward Island', 'Newfoundland and Labrador', 'Louisiana', 'Ontario', 'Oregon', 'New Brunswick', 'Nebraska', 'Georgia', 'Delaware', 'Virginia', 'New Hampshire', 'Florida', 'North Carolina', 'Indiana', 'Minnesota']


### timezone mappings

In [69]:
import pytz


city_timezone_mapping = {
    'Kingston': 'America/Toronto',  # Canada (Eastern Time)
    'Calgary': 'America/Edmonton',  # Canada (Mountain Time)
    'Bethesda': 'America/New_York',  # USA (Eastern Time)
    'Nashville': 'America/Chicago',  # USA (Central Time)
    'Saint Paul': 'America/Chicago',  # USA (Central Time)
    'Berkeley': 'America/Los_Angeles',  # USA (Pacific Time)
    'Little Rock': 'America/Chicago',  # USA (Central Time)
    'Sunnyvale': 'America/Los_Angeles',  # USA (Pacific Time)
    'Salinas': 'America/Los_Angeles',  # USA (Pacific Time)
    'Ann Arbor': 'America/Detroit',  # USA (Eastern Time)
    'Auburn': 'America/New_York',  # USA (Eastern Time)
    'Germantown': 'America/Chicago',  # USA (Central Time)
    'Fairfield': 'America/New_York',  # USA (Eastern Time)
    'Charlotte': 'America/New_York',  # USA (Eastern Time)
    'Guelph': 'America/Toronto',  # Canada (Eastern Time)
    'Vancouver': 'America/Vancouver',  # Canada (Pacific Time)
    'Stamford': 'America/New_York',  # USA (Eastern Time)
    'Pleasanton': 'America/Los_Angeles',  # USA (Pacific Time)
    'Cambridge': 'America/New_York',  # USA (Eastern Time)
    'Paramus': 'America/New_York',  # USA (Eastern Time)
    'Jacksonville': 'America/New_York',  # USA (Eastern Time)
    'Livermore': 'America/Los_Angeles',  # USA (Pacific Time)
    'Eau Claire': 'America/Chicago',  # USA (Central Time)
    'Sacramento': 'America/Los_Angeles',  # USA (Pacific Time)
    'Los Altos': 'America/Los_Angeles',  # USA (Pacific Time)
    'Dartmouth': 'America/Halifax',  # Canada (Atlantic Time)
    'Everett': 'America/Los_Angeles',  # USA (Pacific Time)
    'Modesto': 'America/Los_Angeles',  # USA (Pacific Time)
    'Saint John': 'America/Halifax',  # Canada (Atlantic Time)
    'Lethbridge': 'America/Edmonton',  # Canada (Mountain Time)
    'Kansas City': 'America/Chicago',  # USA (Central Time)
    'Madison': 'America/Chicago',  # USA (Central Time)
    'Evanston': 'America/Chicago',  # USA (Central Time)
    'Fontana': 'America/Los_Angeles',  # USA (Pacific Time)
    'Hayward': 'America/Los_Angeles',  # USA (Pacific Time)
    'Franklin': 'America/Chicago',  # USA (Central Time)
    'Somerville': 'America/New_York',  # USA (Eastern Time)
    'Chantilly': 'America/New_York',  # USA (Eastern Time)
    'Halifax': 'America/Halifax',  # Canada (Atlantic Time)
    'Laredo': 'America/Chicago',  # USA (Central Time)
    'Windsor': 'America/Toronto',  # Canada (Eastern Time)
    'Richardson': 'America/Chicago',  # USA (Central Time)
    'Oshawa': 'America/Toronto',  # Canada (Eastern Time)
    'Denver': 'America/Denver',  # USA (Mountain Time)
    'Cherry Hill': 'America/New_York',  # USA (Eastern Time)
    'Fredericton': 'America/Moncton',  # Canada (Atlantic Time)
    'Houston': 'America/Chicago',  # USA (Central Time)
    'Regina': 'America/Regina',  # Canada (Central Time)
    'Buffalo': 'America/New_York',  # USA (Eastern Time)
    'Coffeyville': 'America/Chicago',  # USA (Central Time)
    'Sugar Land': 'America/Chicago',  # USA (Central Time)
    'Johns Creek': 'America/New_York',  # USA (Eastern Time)
    'Winston-Salem': 'America/New_York',  # USA (Eastern Time)
    'Fargo': 'America/Chicago',  # USA (Central Time)
    'Granby': 'America/Toronto',  # Canada (Eastern Time)
    'New Orleans': 'America/Chicago',  # USA (Central Time)
    'Hartford': 'America/New_York',  # USA (Eastern Time)
    'Bridgeport': 'America/New_York',  # USA (Eastern Time)
    'Reno': 'America/Los_Angeles',  # USA (Pacific Time)
    'Raleigh': 'America/New_York',  # USA (Eastern Time)
    'Miami': 'America/New_York',  # USA (Eastern Time)
    'Chicago': 'America/Chicago',  # USA (Central Time)
    'San Francisco': 'America/Los_Angeles',  # USA (Pacific Time)
    'Los Angeles': 'America/Los_Angeles',  # USA (Pacific Time)
    'Philadelphia': 'America/New_York',  # USA (Eastern Time)
    'Anchorage': 'America/Anchorage',  # USA (Alaska Time)
    'Minneapolis': 'America/Chicago',  # USA (Central Time)
    'New York': 'America/New_York',  # USA (Eastern Time)
    'Boston': 'America/New_York',  # USA (Eastern Time)
    'Dallas': 'America/Chicago',  # USA (Central Time)
    'Boulder': 'America/Denver',  # USA (Mountain Time)
    'Seattle': 'America/Los_Angeles',  # USA (Pacific Time)
    'Montgomery': 'America/Chicago',  # USA (Central Time)
    'Phoenix': 'America/Phoenix',  # USA (Mountain Standard Time, no DST)
    'Las Vegas': 'America/Los_Angeles',  # USA (Pacific Time)
}


region_timezone_dict = {
    'Texas': 'America/Chicago',           # Central Time
    'Nova Scotia': 'America/Halifax',     # Atlantic Time
    'Wisconsin': 'America/Chicago',       # Central Time
    'Massachusetts': 'America/New_York',  # Eastern Time
    'Nebraska': 'America/Chicago',        # Central Time
    'Wyoming': 'America/Denver',          # Mountain Time
    'Kentucky': 'America/New_York',       # Eastern Time
    'Maryland': 'America/New_York',       # Eastern Time
    'Arkansas': 'America/Chicago',        # Central Time
    'Saskatchewan': 'America/Regina',     # Central Time (no DST)
    'South Dakota': 'America/Chicago',    # Central Time
    'Ohio': 'America/New_York',           # Eastern Time
    'Nevada': 'America/Los_Angeles',      # Pacific Time
    'Kansas': 'America/Chicago',          # Central Time
    'Virginia': 'America/New_York',       # Eastern Time
    'Minnesota': 'America/Chicago',       # Central Time
    'Washington': 'America/Los_Angeles',  # Pacific Time
    'Maine': 'America/New_York',          # Eastern Time
    'Vermont': 'America/New_York',        # Eastern Time
    'Alaska': 'America/Anchorage',        # Alaska Time
    'Manitoba': 'America/Winnipeg',       # Central Time
    'Alabama': 'America/Chicago',         # Central Time
    'Iowa': 'America/Chicago',            # Central Time
    'Rhode Island': 'America/New_York',   # Eastern Time
    'Missouri': 'America/Chicago',        # Central Time
    'Hawaii': 'Pacific/Honolulu',         # Hawaii-Aleutian Time (no DST)
    'Florida': 'America/New_York',        # Eastern Time (some parts Central)
    'Michigan': 'America/Detroit',        # Eastern Time
    'Tennessee': 'America/Chicago',       # Central Time (some parts Eastern)
    'Pennsylvania': 'America/New_York',   # Eastern Time
    'Delaware': 'America/New_York',       # Eastern Time
    'Prince Edward Island': 'America/Halifax',  # Atlantic Time
    'District of Columbia': 'America/New_York', # Eastern Time
    'North Dakota': 'America/Chicago',    # Central Time
    'Oklahoma': 'America/Chicago',        # Central Time
    'New Brunswick': 'America/Halifax',   # Atlantic Time
    'Ontario': 'America/Toronto',         # Eastern Time
    'Quebec': 'America/Toronto',          # Eastern Time
    'Idaho': 'America/Boise',             # Mountain Time
    'Indiana': 'America/Indiana/Indianapolis',  # Eastern Time
    'New York': 'America/New_York',       # Eastern Time
    'Mississippi': 'America/Chicago',     # Central Time
    'Georgia': 'America/New_York',        # Eastern Time
    'Illinois': 'America/Chicago',        # Central Time
    'Louisiana': 'America/Chicago',       # Central Time
    'New Jersey': 'America/New_York',     # Eastern Time
    'West Virginia': 'America/New_York',  # Eastern Time
    'South Carolina': 'America/New_York', # Eastern Time
    'Alberta': 'America/Edmonton',        # Mountain Time
    'Arizona': 'America/Phoenix',         # Mountain Time (no DST)
    'North Carolina': 'America/New_York', # Eastern Time
    'Newfoundland and Labrador': 'America/St_Johns',  # Newfoundland Time
    'California': 'America/Los_Angeles',  # Pacific Time
    'Utah': 'America/Denver',             # Mountain Time
    'New Hampshire': 'America/New_York',  # Eastern Time
    'Connecticut': 'America/New_York',    # Eastern Time
    'Montana': 'America/Denver',          # Mountain Time
    'British Columbia': 'America/Vancouver',  # Pacific Time
    'Colorado': 'America/Denver',         # Mountain Time
    'New Mexico': 'America/Denver',       # Mountain Time
    'Oregon': 'America/Los_Angeles'       # Pacific Time
}



## timezone conversion

In [70]:

# Function to convert the UTC datetime to local time and extract date, hour, and minute
def extract_local_date_hour_minute(row, region_timezone_dict):
    # Check if the row contains a valid timestamp
    if pd.isnull(row['event_timestamp']):
        return pd.NaT, pd.NaT, pd.NaT  # Return NaT for all if the timestamp is null

    # Get the region and corresponding timezone
    region = row['geo_region']
    tz = region_timezone_dict.get(region, None)  # Fetch timezone, fallback to None
    
    # If a valid timezone is found, apply timezone conversion
    if tz:
        try:
            local_time = row['event_timestamp'].tz_convert(tz)  # Convert to local time
            
            local_date = local_time.date()  # Extract local date
            local_hour = local_time.hour  # Extract local hour
            local_minute = local_time.minute  # Extract local minute
            
            return local_date, local_hour, local_minute
        except Exception as e:
            print(f"Error converting to timezone {tz}: {e}")
            return pd.NaT, pd.NaT, pd.NaT
    else:
        # Fallback to UTC timestamp
        return row['event_timestamp'].date(), row['event_timestamp'].hour, row['event_timestamp'].minute

# Apply the conversion function to the DataFrame
data[['local_date', 'local_hour', 'local_minute']] = data.apply(
    lambda row: extract_local_date_hour_minute(row, region_timezone_dict), axis=1, result_type='expand'
)

# Convert local_date to datetime format
data['local_date'] = pd.to_datetime(data['local_date'])




### additional date columns creation

In [71]:
data['hour_minute_fraction'] = round(data['local_hour'] + data['local_minute'] / 60,2)  # Hour + fraction of minute

# Categorize the time of day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

data['time_of_day'] = data['local_hour'].apply(categorize_time_of_day)



# create year, quarter, month, day number of week, weekend/weekday based on event_date column

# Create new columns
data['year'] = data['local_date'].dt.year
data['quarter'] = data['local_date'].dt.quarter
data['month'] = data['local_date'].dt.month
data['day'] = data['local_date'].dt.day
data['day_of_week'] = data['local_date'].dt.day_name()  
data['day_of_year'] = data['local_date'].dt.dayofyear  # Day of the year
data['week_number'] = data['local_date'].dt.isocalendar().week  # ISO week number

# Assuming 'local_date' is in datetime format, otherwise you can parse it using pd.to_datetime
def week_of_month(dt):
    first_day = dt.replace(day=1)
    # Calculate the week of the month by comparing the current date to the first day of the month
    return (dt.day + first_day.weekday()) // 7 + 1

# Apply this function to your 'local_date' column
data['week_of_month'] = data['local_date'].apply(week_of_month)



data['day_of_week_nr'] = data['local_date'].dt.weekday  # Monday=0, Sunday=6
data['is_weekend'] = data['day_of_week_nr'].apply(lambda x: 1 if x >= 5 else 0)  # 1 for weekend, 0 for weekday
data.drop(['day_of_week_nr'],axis=1) # already symbolic. not needed





,event_date,session_id,user_pseudo_id,event_name,event_timestamp,page_location,page_title,device_category,device_mobile_brand_name,device_mobile_model_name,...,hour_minute_fraction,time_of_day,year,quarter,month,day,day_of_week,day_of_year,week_number,is_weekend
0,2020-11-25,9917649834,5.160882e+07,page_view,2020-11-25 06:46:46.920242+00:00,https://shop.googlemerchandisestore.com/Google...,New | Google Merchandise Store,mobile,Apple,iPhone,...,1.77,Night,2020,4,11,25,Wednesday,330,48,0
1,2020-11-18,4591484845,4.313977e+07,view_item,2020-11-18 17:43:24.803054+00:00,https://shop.googlemerchandisestore.com/Google...,Kids | Apparel | Google Merchandise Store,mobile,Apple,iPhone,...,12.72,Afternoon,2020,4,11,18,Wednesday,323,47,0
2,2020-12-04,1078908601,6.044075e+07,scroll,2020-12-04 02:54:53.967285+00:00,https://shop.googlemerchandisestore.com/Google...,Google Men's Softshell Moss,desktop,Mozilla,Firefox,...,21.90,Night,2020,4,12,3,Thursday,338,49,0
3,2020-12-03,2627211832,6.822684e+06,view_item,2020-12-03 03:15:37.461563+00:00,https://shop.googlemerchandisestore.com/Google...,Men's / Unisex | Apparel | Google Merchandise ...,desktop,Google,ChromeBook,...,19.25,Evening,2020,4,12,2,Wednesday,337,49,0
4,2020-12-15,5511448088,3.736575e+07,add_to_cart,2020-12-15 10:36:09.110030+00:00,https://shop.googlemerchandisestore.com/Google...,Small Goods | Lifestyle | Google Merchandise S...,mobile,Samsung,<Other>,...,2.60,Night,2020,4,12,15,Tuesday,350,51,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340140,2020-12-05,321756288,2.400522e+07,page_view,2020-12-05 03:33:08.859454+00:00,https://shop.googlemerchandisestore.com/Google...,Drinkware | Lifestyle | Google Merchandise Store,mobile,Huawei,<Other>,...,22.55,Night,2020,4,12,4,Friday,339,49,0
340141,2021-01-15,4306560611,2.997664e+06,page_view,2021-01-15 10:51:49.855156+00:00,https://shop.googlemerchandisestore.com/,Home,mobile,<Other>,<Other>,...,5.85,Morning,2021,1,1,15,Friday,15,2,0
340142,2020-12-08,5679840314,6.825749e+06,page_view,2020-12-08 10:50:48.590289+00:00,https://www.googlemerchandisestore.com/,Google Online Store,mobile,Apple,iPhone,...,2.83,Night,2020,4,12,8,Tuesday,343,50,0
340143,2020-12-10,3563782843,1.124405e+07,scroll,2020-12-10 03:06:49.905906+00:00,https://shop.googlemerchandisestore.com/store....,Home,mobile,Apple,iPhone,...,22.10,Night,2020,4,12,9,Wednesday,344,50,0


# device columns

- for many cases we assumed devices, brands and os versions of 2021 as top devices

## device mobile brand

In [72]:

# Fill 'device_mobile_brand_name' with 'PC' where the conditions are met
data.loc[(data['device_operating_system'] == 'Windows') & (data['device_category'] == 'desktop'), 'device_mobile_brand_name'] = 'PC'
data.loc[(data['device_operating_system'] == 'Web') & (data['device_category'] == 'desktop') & ((data['device_mobile_model_name'].isin(['Chrome','Edge','Firefox']))), 'device_mobile_brand_name'] = 'PC'
data.loc[(data['device_mobile_brand_name'] == 'Microsoft') & (data['device_category'] == 'desktop'), 'device_mobile_brand_name'] = 'PC'


# data['device_mobile_brand_name'].value_counts()

## device_mobile_model_name 

In [73]:


data.loc[(data['device_mobile_brand_name'] == 'Samsung'), 'device_mobile_model_name'] = 'Galaxy S21'

data.loc[(data['device_mobile_brand_name'] == 'Xiaomi'), 'device_mobile_model_name'] = 'Mi 11'

data.loc[(data['device_mobile_brand_name'] == 'Huawei') & (data['device_category'] == 'mobile'), 'device_mobile_model_name'] = 'P50'

data.loc[(data['device_mobile_brand_name'] == 'Apple') & (data['device_category'] == 'desktop'), 'device_mobile_model_name'] = 'Macintosh'

data.loc[(data['device_mobile_brand_name'] == 'Apple') & (data['device_category'] == 'mobile'), 'device_mobile_model_name'] = 'iPhone'
data.loc[(data['device_mobile_brand_name'] == 'Apple') & (data['device_category'] == 'tablet'), 'device_mobile_model_name'] = 'iPad'


data.loc[(data['device_mobile_brand_name'] == 'PC')& (data['device_mobile_model_name'] == 'Chrome'), 'device_mobile_model_name'] = 'PC'



data['device_mobile_model_name'] = (
    data['device_mobile_brand_name'] + " - " +
    data['device_mobile_model_name'].astype(str).where(pd.notna(data['device_mobile_model_name']), '')
)


if test_data==True:
    data['device_mobile_model_name'].value_counts()

## device_operating_system

In [74]:

# Chrome OS - ensure 'desktop' is correctly spelled
data.loc[(data['device_mobile_model_name'] == 'ChromeBook') & (data['device_category'] == 'desktop'), 'device_operating_system'] = 'ChromeOS'

# iOS - for iPhone and iPad
data.loc[data['device_mobile_model_name'].isin(['iPhone', 'iPad']) | (data['device_mobile_brand_name'] == 'Apple') | ((data['device_mobile_model_name'] == 'Apple') & (data['device_category'].isin(['mobile','tablet']))), 'device_operating_system'] = 'iOS'

# Android - for specified brands
android_brands = ['Xiaomi', 'Huawei', 'Samsung']
data.loc[data['device_mobile_brand_name'].isin(android_brands), 'device_operating_system'] = 'Android'
data.loc[(data['device_mobile_brand_name'] == 'Google') & (data['device_category'].isin(['mobile','tablet'])), 'device_operating_system'] = 'Android'

# macOS
data.loc[(data['device_mobile_brand_name'] == 'Apple') & (data['device_category'] == 'desktop'), 'device_operating_system'] = 'MacOS'

# Windows
data.loc[(data['device_operating_system'] == 'Web') & (data['device_category'] == 'desktop') & ((data['device_mobile_brand_name'] == 'PC')), 'device_operating_system'] = 'Windows'
data.loc[(data['device_operating_system'] == 'Web') & (data['device_category'] == 'desktop') & ((data['device_mobile_brand_name'] == 'Mozilla')), 'device_operating_system'] = 'Windows'
data.loc[(data['device_category'] == 'desktop') & ((data['device_mobile_brand_name'] == 'Microsoft')), 'device_operating_system'] = 'Windows'

if test_data==True:
    data.groupby(['device_category','device_operating_system', 'device_mobile_brand_name']).agg(
        unique_event_count=('event_timestamp', 'nunique')
    ).sort_values('unique_event_count',ascending=False).reset_index()


## device_operating_system_version

In [75]:
# replace all string characters and keep float values
data['device_operating_system_version'] = data['device_operating_system_version'].str.extract(r'(\d+\.\d+|\d+)')


In [76]:
# for chrome os consider same browser version
# https://chromereleases.googleblog.com/2021/
data.loc[(data['device_operating_system'] == 'ChromeOS') & (data['device_operating_system_version'].isnull()), 'device_operating_system_version'] = data['device_web_info_browser_version']


data['device_operating_system_version'] = (
    data['device_operating_system'] + " - " +
    data['device_operating_system_version'].astype(str).where(pd.notna(data['device_operating_system_version']), '')
)


if test_data==True:
    data['device_operating_system_version'].value_counts()


In [77]:
if test_data==True:
    data['device_operating_system_version'].value_counts()

## device_language

In [78]:

if test_data==True:
    data.groupby(['geo_country','device_language']).agg(
        unique_event_count=('event_timestamp', 'nunique')
    ).sort_values('unique_event_count',ascending=False).reset_index()

In [79]:
if test_data==True:
    data.groupby(['geo_country']).agg(
        null_device_language_count=('device_language', lambda x: x.isna().sum())
    ).sort_values('null_device_language_count',ascending=False).reset_index()

In [80]:
if test_data==True:
    data['device_language'].value_counts()

## device_web_info_browser

In [81]:
data.loc[(data['device_web_info_browser'] == 'Android Webview'), 'device_web_info_browser'] = "Chrome"

if test_data==True:
    data['device_web_info_browser'].value_counts()

## device_web_info_browser_version

let us concatenate so that the values make sense and not mixed between browser versions

In [82]:


data['device_web_info_browser_version'] = (
    data['device_web_info_browser'] + ' - ' +
    data['device_web_info_browser_version'].astype(str).where(pd.notna(data['device_web_info_browser_version']), '')
)


if test_data==True:
    data['device_operating_system_version'].value_counts()

# Traffic Columns

## session counts

In [83]:
if test_data==True:
    data.groupby(['traffic_source_medium','traffic_source_source']).agg(
    unique_session_count=('session_id', 'nunique')
    ).sort_values('unique_session_count',ascending=False).reset_index()

## remove parenthesis ()

In [84]:
data['traffic_source_source'] = data['traffic_source_source'].str.replace(r'\(|\)', '', regex=True).str.strip()
data['traffic_source_medium'] = data['traffic_source_medium'].str.replace(r'\(|\)', '', regex=True).str.strip()

if test_data==True:
    data.groupby(['traffic_source_medium','traffic_source_source']).agg(
        unique_session_count=('session_id', 'nunique')
    ).sort_values('unique_session_count',ascending=False).reset_index()

## Replace values

- bad referral naming from shop.googlemerchandisestore.com means badly tracked and we should consider direct traffic instead
- medium =none is referring to direct traffic and we will use the same name to not confuse with null values
- data deleted is most likely paid campaign by google to avoid confidential data exposure so we will replace with cpc / google as well


In [85]:
# source is merchandisestore.com then direct (bad parameter)
data.loc[(data['traffic_source_source'] == 'shop.googlemerchandisestore.com') & (data['traffic_source_medium'] == 'referral'), ['traffic_source_medium', 'traffic_source_source']] = ['direct','direct']


# referral traffic
data.loc[(data['traffic_source_medium'] == 'referral'), ['traffic_source_medium', 'traffic_source_source']] = ['referral','referral_link']

# google organic
data.loc[(data['traffic_source_source'].isnull()) & (data['traffic_source_medium']== 'organic'),  ['traffic_source_medium', 'traffic_source_source']] = ['organic','google']


# when we have direct traffic it is direct traffic
data.loc[(data['traffic_source_source'] == 'direct'), 'traffic_source_medium'] = 'direct'

# data deleted is paid campaign cpc by google
data.loc[(data['traffic_source_source'] == 'data deleted') | (data['traffic_source_medium'] == 'data deleted'), ['traffic_source_medium', 'traffic_source_source']] = ['cpc', 'google']


# full null values are direct
data.loc[(data['traffic_source_source'].isnull()) & (data['traffic_source_medium'].isnull()),  ['traffic_source_medium', 'traffic_source_source']] = ['direct','direct']
# data.loc[(data['traffic_source_source'].isnull()) & (data['traffic_source_medium'].isnull()), 'traffic_source_medium'] = 'direct'

if test_data == True:
    data.groupby(['traffic_source_source','traffic_source_medium']).agg(
        unique_session_count=('session_id', 'nunique')
    ).sort_values('unique_session_count',ascending=False).reset_index()

# Page columns

## pagelocation string work

In [86]:

# remove page unavailable
data = data.loc[data['page_title'] != 'Page Unavailable']


# lowercase everything to guarantee string match
data['page_location'] = data['page_location'].str.lower()
data['page_title'] = data['page_title'].str.lower()


# domain readability
data['page_location'] = data['page_location'].str.replace(r'shop.googlemerchandisestore.com/store.html', 'shop.googlemerchandisestore.com/').str.strip()
data['page_location'] = data['page_location'].str.replace(r'+', ' ').str.strip()
data['page_location'] = data['page_location'].str.replace(r'https://', '').str.strip()
data['page_location'] = data['page_location'].str.replace(r'http://', '').str.strip()
data['page_location'] = data['page_location'].str.replace(r'www.', '').str.strip()





## remove low percentile page record count

In [87]:
df_pages_agg=data.groupby(['page_location']).agg(
    total_events=('event_timestamp', 'count'),
).sort_values('total_events',ascending=False).reset_index()


# Calculate the percentile of event counts
event_count_percentile = df_pages_agg['total_events'].quantile(0.30)

print(f"low percentile of event count: {event_count_percentile}")

# # Filter out page paths below the 10th percentile
df_pages_agg_filtered = df_pages_agg[df_pages_agg['total_events'] >= event_count_percentile]



data=data.merge(df_pages_agg_filtered[['page_location']],
                on=['page_location'],
                how='inner')



low percentile of event count: 7.0


## split categories in page paths

In [88]:
# Step 1: Split 'page_location' into 4 parts (max)
split_columns = data['page_location'].str.split('/', n=4, expand=True)

# Step 2: Assign the first three parts to new columns (ignore the first empty part if there is a leading '/')
data['domain'] = split_columns[0] # url_domain 
data['page_path_level_1'] = split_columns[1].replace('', pd.NA)
data['page_path_level_2'] = split_columns[2].replace('', pd.NA)
data['page_path_level_3'] = split_columns[3].replace('', pd.NA)

# Count the number of non-null levels in the path
data['path_length'] = data[['page_path_level_1', 'page_path_level_2', 'page_path_level_3']].notna().sum(axis=1)



df_pages_total=data[['page_title','page_location','page_path_level_1','page_path_level_2','page_path_level_3','path_length']]


## fill length page path with page title

In [89]:
data['page_path_level_1'] = data.apply(
    lambda row: row['page_title'] if pd.isna(row['page_path_level_2']) else row['page_path_level_1'], 
    axis=1
)

df_pages_total=data[['page_title','page_location','page_path_level_1','page_path_level_2','page_path_level_3']]


## page location long tail conversion to "other"

In [90]:
# Step 1: Aggregate events by 'page_location' and 'page_path_level_3'
df_pages_agg = data.groupby(['page_location', 'page_path_level_3']).agg(
    total_events=('event_timestamp', 'count'),
).sort_values('total_events', ascending=False).reset_index()

# Step 2: Calculate the desired percentile of total events to consider long tail
event_count_percentile = df_pages_agg['total_events'].quantile(0.75)

# Step 3: Create a mask for long tail pages
long_tail_mask = df_pages_agg['total_events'] < event_count_percentile

# Step 4: Create a dictionary to map original page_path_level_3 to "other"
long_tail_mapping = dict(zip(df_pages_agg['page_path_level_3'], 
                             df_pages_agg['page_path_level_3'].where(~long_tail_mask, "other")))


# Step 5: Replace the values directly in the original DataFrame
# Only apply the mapping if path_length is 3
data['page_path_level_3'] = data.apply(
    lambda row: long_tail_mapping.get(row['page_path_level_3'], row['page_path_level_3']) if row['path_length'] == 3 else row['page_path_level_3'],
    axis=1
)



## fill the other page path levels with previous page path column

This will allow for hierarchical encoding without sacrificing columns or rows

In [91]:
# Fill 'page_path_level_2' by concatenating 'page_path_1' and 'page_path_2' (if 'page_path_2' is null)
data['page_path_level_2'] = data.apply(
    lambda row: row['page_path_level_1'] if pd.isna(row['page_path_level_2']) 
    else f"{row['page_path_level_1']}/{row['page_path_level_2']}", axis=1
)

# Fill 'page_path_level_3' by concatenating 'page_path_level_2' and 'page_path_3' (if 'page_path_3' is null)
data['page_path_level_3'] = data.apply(
    lambda row: row['page_path_level_2'] if pd.isna(row['page_path_level_3']) 
    else f"{row['page_path_level_2']}/{row['page_path_level_3']}", axis=1
)


# Final df without relevant columns

In [92]:
data_final=data.drop(['page_title','page_location','session_id','user_pseudo_id','event_timestamp','local_date','event_date'],axis=1) 

In [93]:
data.shape

(307986, 42)

In [94]:
data_final.to_csv('data/df_merch_pre_proc.csv',index=False)

# df_merch_pre_proc.csv
# df_merch_profile.csv
# df_merch_data_prep.csv

# Excel File for encoding mapping

In [95]:
import os
import pandas as pd
from openpyxl import load_workbook


# Function to append distinct combinations of selected columns into sheets in an Excel file
def append_columns_to_excel(df, columns_dict, output_file):
    """
    Append distinct combinations of selected columns into separate sheets in an existing Excel file,
    with the columns ordered by their names for easier hierarchical encoding and add an empty encoding column.

    Args:
    df (pd.DataFrame): The DataFrame containing the columns to save.
    columns_dict (dict): Dictionary where keys are sheet names, and values are lists of column names to include.
    output_file (str): The path of the Excel file to save the sheets.

    Returns:
    None
    """
    # Check if the file exists and is a valid Excel file
    if os.path.exists(output_file):
        try:
            # Try to load the existing workbook
            with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                writer.book = load_workbook(output_file)
                
                # Loop over each sheet name and corresponding list of columns
                for sheet_name, columns in columns_dict.items():
                    # Check if all the specified columns exist in the DataFrame
                    missing_columns = [col for col in columns if col not in df.columns]
                    if missing_columns:
                        print(f"Warning: The following columns are not found in the DataFrame for sheet '{sheet_name}': {missing_columns}")
                        continue

                    # Get distinct combinations of the selected columns
                    distinct_values = df[columns].drop_duplicates().dropna(how='all')

                    # Sort distinct values by the specified columns for hierarchical grouping
                    distinct_values.sort_values(by=columns, inplace=True)

                    # Add an empty encoding column for each column in the DataFrame
                    for col in columns:
                        distinct_values[f'{col}_enc'] = pd.NA

                    # Write distinct values to a new sheet named after the sheet_name
                    distinct_values.to_excel(writer, sheet_name=sheet_name, index=False)
                    
        except Exception as e:
            print(f"Error: {e}")
            print("The file might be corrupt or invalid. Creating a new file.")
            # Create a new file if loading fails
            with pd.ExcelWriter(output_file, engine='openpyxl', mode='w') as writer:
                for sheet_name, columns in columns_dict.items():
                    missing_columns = [col for col in columns if col not in df.columns]
                    if missing_columns:
                        print(f"Warning: The following columns are not found in the DataFrame for sheet '{sheet_name}': {missing_columns}")
                        continue
                    distinct_values = df[columns].drop_duplicates().dropna(how='all')
                    distinct_values.sort_values(by=columns, inplace=True)  # Sort distinct values

                    # Add an empty encoding column for each column in the DataFrame
                    for col in columns:
                        distinct_values[f'{col}_enc'] = pd.NA

                    distinct_values.to_excel(writer, sheet_name=sheet_name, index=False)

    else:
        # If the file does not exist, create a new one
        with pd.ExcelWriter(output_file, engine='openpyxl', mode='w') as writer:
            for sheet_name, columns in columns_dict.items():
                missing_columns = [col for col in columns if col not in df.columns]
                if missing_columns:
                    print(f"Warning: The following columns are not found in the DataFrame for sheet '{sheet_name}': {missing_columns}")
                    continue
                distinct_values = df[columns].drop_duplicates().dropna(how='all')
                distinct_values.sort_values(by=columns, inplace=True)  # Sort distinct values

                # Add an empty encoding column for each column in the DataFrame
                for col in columns:
                    distinct_values[f'{col}_enc'] = pd.NA

                distinct_values.to_excel(writer, sheet_name=sheet_name, index=False)
                

# Define the groups of columns for hierarchical encoding, grouped by sheet name
columns_to_save = {
    'event_name' : ['event_name'],  
    'device_category': ['device_category'],   
    'device_mobile_brand_name': ['device_mobile_brand_name'],
    'device_mobile_model_name': ['device_mobile_model_name'],
    'device_language': ['device_language'],
    'device_category': ['device_category'],      
    'device_operating_system_version': ['device_operating_system_version'],   
    'device_operating_system': ['device_operating_system'],   
    'device_web_info_browser': ['device_web_info_browser'],   
    'device_web_info_browser_version': ['device_web_info_browser_version'],
    'geo_country': ['geo_country'],   
    'traffic_source_medium':['traffic_source_medium'],
    'traffic_source_source':['traffic_source_source'],
    'page_path_level_1':['page_path_level_1'],
    'page_path_level_2':['page_path_level_2'],
    'page_path_level_3':['page_path_level_3'],
    
}

# Save the distinct values combinations of each column group into corresponding sheets
append_columns_to_excel(data_final, columns_to_save, f'data/df_merch_values_pre_encoding.xlsx')

print("Excel file has been updated successfully.")

Error: property 'book' of 'OpenpyxlWriter' object has no setter
The file might be corrupt or invalid. Creating a new file.
Excel file has been updated successfully.
